In [127]:
import pandas as pd
from urllib.request import urlopen,Request
from urllib.error import HTTPError,URLError
from io import StringIO
import json
import requests

In [22]:
codes = pd.read_fwf('./Codes.txt')
stkcd = []
firm = []
for v in codes.iloc[:,0]:
    code = v.split(sep=',')[0]
    name = v.split(sep=',')[1]
    stkcd.append(code)
    firm.append(name)
stkcd_name = dict(zip(stkcd,firm))
stkcd_name

{'000002': '万科A',
 '000003': 'PT 金田A',
 '000004': '国农科技',
 '000005': '世纪星源',
 '000006': '深振业A',
 '000007': '全新好',
 '000008': '神州高铁',
 '000009': '中国宝安',
 '000010': '美丽生态',
 '000011': '深物业A',
 '000012': '南玻A',
 '000013': '*ST 石化A',
 '000014': '沙河股份',
 '000015': 'PT 中浩A',
 '000016': '深康佳A',
 '000017': '深中华A',
 '000018': '神州长城',
 '000019': '深深宝A',
 '000020': '深华发A',
 '000021': '深科技',
 '000022': '深赤湾A',
 '000023': '深天地A',
 '000024': '招商地产',
 '000025': '特力A',
 '000026': '飞亚达A',
 '000027': '深圳能源',
 '000028': '国药一致',
 '000029': '深深房A',
 '000030': '富奥股份',
 '000031': '中粮地产',
 '000032': '深桑达A',
 '000033': '新都退',
 '000034': '神州数码',
 '000035': '中国天楹',
 '000036': '华联控股',
 '000037': '深南电A',
 '000038': '深大通',
 '000039': '中集集团',
 '000040': '东旭蓝天',
 '000042': '中洲控股',
 '000043': '中航善达',
 '000045': '深纺织A',
 '000046': '泛海控股',
 '000047': 'ST 中侨',
 '000048': '*ST 康达',
 '000049': '德赛电池',
 '000050': '深天马A',
 '000055': '方大集团',
 '000056': '皇庭国际',
 '000058': '深赛格',
 '000059': '华锦股份',
 '000060': '中金岭南',
 '000061':

In [154]:
def getReportInfo(url,token,code,start="20120101",end="20171231"):
    post_data = "scode=%s&sdate=%s&edate=%s&access_token=%s"%(code,start,end,token)
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
    try:
        print("提取%s公司报告信息..." % code)
        req = Request(url,headers=headers)
        html = urlopen(req,StringIO(post_data))
    except(HTTPError,URLError) as e:
        return pd.DataFrame()
    content = html.read().decode('utf-8')
    json_file = json.loads(content)
    info_df = pd.DataFrame(json_file['records'])
    return info_df

url = "http://webapi.cninfo.com.cn/api/info/p_info3015"
token = "2de97e042f4cb8ce97379e5876809cc1"

report_info = pd.DataFrame()
c = 0
for code in stkcd:
    info_df = getReportInfo(url,token,code)
    c += 1
    print("已提取第%s[第%s家]公司报告信息" %(code,c))
    report_info = pd.concat([report_info,info_df])
    if c==len(stkcd):
        print("提取完毕")

提取000002公司报告信息...
已提取第000002[第1家]公司报告信息
提取000003公司报告信息...
已提取第000003[第2家]公司报告信息
提取000004公司报告信息...
已提取第000004[第3家]公司报告信息
提取000005公司报告信息...
已提取第000005[第4家]公司报告信息
提取000006公司报告信息...
已提取第000006[第5家]公司报告信息
提取000007公司报告信息...
已提取第000007[第6家]公司报告信息
提取000008公司报告信息...
已提取第000008[第7家]公司报告信息
提取000009公司报告信息...
已提取第000009[第8家]公司报告信息
提取000010公司报告信息...
已提取第000010[第9家]公司报告信息
提取000011公司报告信息...
已提取第000011[第10家]公司报告信息
提取000012公司报告信息...


RemoteDisconnected: Remote end closed connection without response

In [81]:
get_token = "d75dd55f1b1e835a67b7349b350afec4"
client_id = '17796401944'
client_secret = 'jge19911001'

urlRequest = Request('http://static.cninfo.com.cn/finalpage/2017-12-12/1204214758.PDF',headers={"token":get_token})
buffer = urlopen(urlRequest).read()
with open("sample.pdf",'wb') as pdf:
    pdf.write(buffer)

# from requests.auth import HTTPBasicAuth
# auth = HTTPBasicAuth(client_id,client_secret)
# r = requests.post('http://static.cninfo.com.cn/finalpage/2017-12-12/1204214758.PDF',auth=auth)
# with open("sample.txt",'wb') as pdf:
#     pdf.write(r.content)

# 提取新浪财经公司股东大会公告

In [34]:
from urllib.request import urlopen,urlretrieve,Request
from urllib.error import HTTPError,URLError
from bs4 import BeautifulSoup
import re
import os
import csv
import time
import random

csvFile = open('./report_info.csv','w+')
writer = csv.writer(csvFile)
writer.writerow(['stkcd','repdate','reptype','title','url'])

if not os.path.exists("./reports"):
    os.mkdir("./reports")

domain = "http://vip.stock.finance.sina.com.cn"
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

for page_num in range(1,1000):
    print("正在采集第%s页..." %page_num)
    url = domain + "/corp/view/vCB_BulletinGather.php?page=%s" % page_num
    req = Request(url,headers=headers)
    try:
        html = urlopen(req)
        page_content = BeautifulSoup(html.read())
        if page_content.tbody.tr.th.text=='对不起，没有相关记录':
            break
    except(HTTPError,URLError) as e:
        print(e)
        
    for tr in page_content.tbody.findAll("tr"):
        tds = tr.findAll('td')
        if len(tds)==2:
            reptype,repdate = [td.text for td in tds]
        else:
            print("td tag is more than two,may contain more info") 
            reptype,repdate = [td.text for td in tds[:2]]

        a = tr.th.findAll("a")
        if 'href' in a[0].attrs:
            link1 = Request(domain+a[0].attrs['href'],headers=headers)
            html1 = BeautifulSoup(urlopen(link1).read())
            title = html1.title.text
            if "股东大会" in title:
                stkcd = re.findall("\d{6}",title)[0]
                if 'href' in a[1].attrs:
                    file_url = Request(a[1].attrs['href'],headers=headers)
                    buffer = urlopen(file_url).read()
#                     urlretrieve(buffer,r"./reports/"+title+".pdf")
                    with open(r"./reports/"+title+".pdf",'wb') as pdf:
                        pdf.write(buffer)
                    csvRow = [stkcd,repdate,reptype,title,a[1].attrs['href']]
                    writer.writerow(csvRow)
                    time.sleep(3+random.randint(0,2))
csvFile.close()

正在采集第1页...
正在采集第2页...


URLError: <urlopen error [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。>

In [ ]:
# tbody.tr.th.a.text
# for child in page_content.tbody.find({"tr"}).children:  #,{"width":"130"})
#     print(child)
# for child in page_content.tbody.find({"tr"}).descendants:  #,{"width":"130"})
#     print(child)

# for sibling in page_content.tbody.find({"tr"}).next_siblings:  #,{"width":"130"})
#     print(sibling)
# page_content.tbody.find("td").attrs
# page_content.tbody.find("td").attrs['width']

In [16]:
from selenium import webdriver
driver = webdriver.Chrome(executable_path=r"C:\Users\win10\AppData\Local\Google\Chrome\Application\chromedriver.exe")
driver.get("http://vip.stock.finance.sina.com.cn/corp/view/vCB_BulletinGather.php?stock_str=&gg_date=&ftype=0")
driver.implicitly_wait(1)
driver.get_cookies()

[{'domain': '.sina.com.cn',
  'httpOnly': False,
  'name': 'Apache',
  'path': '/',
  'secure': False,
  'value': '110.185.17.77_1546879268.837272'},
 {'domain': '.sina.com.cn',
  'expiry': 2147482801.28341,
  'httpOnly': False,
  'name': 'SINAGLOBAL',
  'path': '/',
  'secure': False,
  'value': '110.185.17.77_1546879268.837269'},
 {'domain': '.sina.com.cn',
  'expiry': 1862239266.711109,
  'httpOnly': False,
  'name': 'U_TRS1',
  'path': '/',
  'secure': False,
  'value': '0000004d.9d207159.5c338122.29212a69'},
 {'domain': '.sina.com.cn',
  'expiry': 1577983268,
  'httpOnly': False,
  'name': 'ULV',
  'path': '/',
  'secure': False,
  'value': '1546879268357:1:1:1::'},
 {'domain': '.sina.com.cn',
  'httpOnly': False,
  'name': 'U_TRS2',
  'path': '/',
  'secure': False,
  'value': '0000004d.9d2b7159.5c338122.650b5dbe'},
 {'domain': 'vip.stock.finance.sina.com.cn',
  'httpOnly': False,
  'name': 'FINANCE2',
  'path': '/',
  'secure': False,
  'value': '04a75e4ddb9c7efc0c453ca0be428c53